In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('sgemm_product.csv')

In [0]:
df2 = df.assign(RunMean= (df['Run1 (ms)'] + df['Run2 (ms)']  + df['Run3 (ms)']  + df['Run4 (ms)'] )/4)
df2 = df2.drop(columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'])

In [0]:

X = df2.iloc[:,:14]
y = df2['RunMean']

In [0]:
from sklearn.preprocessing import StandardScaler
X.iloc[:,:10] = StandardScaler().fit_transform(X.iloc[:,:10])

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


from sklearn.preprocessing import StandardScaler
X.iloc[:,:10] = StandardScaler().fit_transform(X.iloc[:,:10])

In [0]:
X_sel = X[['MWG', 'MDIMC', 'MDIMA', 'KWI','VWN', 'STRM', 'STRN', 'SA', 'SB']] 


In [0]:
y_mean = y.mean()
y_sd = y.std()
y = (y-y_mean)/y_sd

In [0]:
y_train_class = []
for i in y_train:
    if i > -0.40076444381591425:
        x = 1
        y_train_class.append(x)
    else:
        x = 0
        y_train_class.append(x)

y_test_class = []
for i in y_test:
    if i > -0.40076444381591425:
        x = 1
        y_test_class.append(x)
    else:
        x = 0
        y_test_class.append(x)
        
n_samples, n_features = X_train.shape

In [0]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [22]:
LOG_LRtrain = {}
LOG_LRcost = {}

LOG_LRtest = {}
dw_l = []
for lr in [0.5]:   
    weights = np.zeros(n_features)
    LOG_LRtrain['LRtrain_%s' % lr] = []
    LOG_LRtest['LRtest_%s' % lr] = []
    LOG_LRcost['LRcost_%s' % lr] = []
    for iteration in range(0,10):
        lin_y_train = np.dot(X_train, weights)
        y_train_pred = sigmoid(lin_y_train)
        dw = (1/n_samples)*np.dot(X_train.T,(y_train_pred-np.asarray(y_train_class)))
        dw_l.append(dw.sum())
        weights = weights - lr*dw
        
        lin_y_train = np.dot(X_train, weights) 
        y_train_pred = sigmoid(lin_y_train)
        y_train_pred_class = []
        for i in y_train_pred:
            if i > 0.40112868791012124:
                x=1
                y_train_pred_class.append(x)
            else:
                x=0
                y_train_pred_class.append(x)
        
        accuracy_train = (np.asarray(y_train_pred_class) == np.asarray(y_train_class)).sum() / len(y_train_class)
        
        lin_y_test = np.dot(X_test, weights) 
        y_test_pred = sigmoid(lin_y_test)
        y_test_pred_class = []
        for i in y_test_pred:
            if i > 0.40112868791012124:
                x=1
                y_test_pred_class.append(x)
            else:
                x=0
                y_test_pred_class.append(x)
        
        accuracy_test = (np.asarray(y_test_pred_class) == np.asarray(y_test_class)).sum() / len(y_test_class)
        class1_cost = -np.asarray(y_test_class)*np.log((y_test_pred))
        class2_cost = (1-np.asarray(y_test_class))*np.log(1-y_test_pred)
        cost = (class1_cost - class2_cost).sum() / len(y_test_class)
        
        LOG_LRcost['LRcost_%s' % lr].append(cost)
   
        LOG_LRtest['LRtest_%s' % lr].append(accuracy_test)
        LOG_LRtrain['LRtrain_%s' % lr].append(accuracy_train)
    print("The accuracy of test data is :",accuracy_test)
    print("The accuracy of train data is :",accuracy_train)

The accuracy of test data is : 0.6984478476821192
The accuracy of train data is : 0.7000758830022075
